# Assigment 7

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

In [5]:
!pip install pandas
!pip install requests
!pip install PyPDF2
!pip install geopy
!pip install tabula-py
!pip install pdfplumber
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40717 sha256=76b471073a70dbcdccc3537171bc6809898dfb6e882ae5c0058d41fd920d82fe
  Stored in directory: /Users/danielmontjoy/Library/Caches/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


In [8]:
import tabula
import numpy as np
import pandas as pd
import os
import urllib.request, json, csv
import numpy as np
from tqdm import tqdm_notebook as tqdm
import requests
import googlemaps
from datetime import datetime

In [9]:
!pip install JPype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 2.7 MB/s eta 0:00:0000:0100:01


In [63]:
## leemos los datos
url = 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'
por=0.393701 * 72
a=44*por
b=125*por
c=154*por
d=175*por
e=200*por
columns_coordinates = [a, b, c, d,e]
datos = tabula.read_pdf(file_path, columns=columns_coordinates, pages='2-5',pandas_options={'header':None},encoding='utf-8', lattice=False,multiple_tables=True)
datos

[                                                    0             1  \
 0                                   OFICINA DIRECCION  DEPARTAMENTO   
 1   AEROPUERTO CENTRO AÉREO COMERCIAL LOCALES 110 ...          LIMA   
 2         BASE NAVAL AV. CTRMTE.  MORA S/N BASE NAVAL          LIMA   
 3   C.C. CANTA CALLAO AV. ELMER FAUCETT Y ALEJANDR...          LIMA   
 4                               CC CANTA CALLO, LC 13           NaN   
 ..                                                ...           ...   
 59  EL REMANSO CA. LOS ALAMOS MZ C-2 LT. 5 - URB. ...          LIMA   
 60  LA MOLINA AV. LA MOLINA MZ. B LOTE 08, URB. RE...          LIMA   
 61  LA ROTONDA AV. LA FONTANA 371, MZ. A LT. 7, UR...          LIMA   
 62                  RINCONADA AV. LA UNIVERSIDAD 1810          LIMA   
 63            SEPARADORA INDUSTRIAL AV. LA MOLINA 540          LIMA   
 
             2          3  
 0   PROVINCIA   DISTRITO  
 1        LIMA     CALLAO  
 2        LIMA     CALLAO  
 3        LIMA     CAL

In [ ]:
columns_coordinates = [72, 144, 216, 288]  # Replace these with your measured coordinates
url = 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'
datos = tabula.read_pdf(file_path, columns=[0, 1, 2, 3, 4], pages='all',pandas_options={'header':None},encoding='utf-8', lattice=False,multiple_tables=True)
datos

In [58]:
datos = pd.concat(pdf_data, ignore_index=True).iloc[ 2: , : 5,].reset_index(drop=True)
# Se selecciona las filas y columnas con los datos necesarios y se reinicia el índice
#data = data_unida.iloc[ 2: , : 5,].reset_index(drop=True)
# Generamos el nombre de las columnas o de las variables y se logra el DataFrame final
headers = ['ofi','dire','dist','pro','dep']
data.columns = headers
data

,ofi,dire,dist,pro,dep
0,BASE NAVAL AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO,NaN
1,C.C. CANTA CALLAO AV. ELMER FAUCETT Y ALEJANDR...,LIMA,LIMA,CALLAO,NaN
2,"CC CANTA CALLO, LC 13",NaN,NaN,NaN,NaN
3,CALLAO AV. SAENZ PEÑA 323,LIMA,LIMA,CALLAO,NaN
4,COLONIAL CALLE OMEGA 149 PARQUE INDUSTRIAL DEL...,LIMA,LIMA,CALLAO,NaN
...,...,...,...,...,...
363,NaN,NaN,CORONEL,NaN,NaN
364,7 DE JUNIO JIRÓN 7 DE JUNIO N.- 201,UCAYALI,PORTILLO,NaN,CALLERIA
365,CORONELPUCALLPA ESQUINA UCAYALI Y RAYMONDI 699,UCAYALI,PORTILLO,NaN,CALLERIA
366,"AV. CENTENARIO 2086 Y 2192, JR. LAUREANO",NaN,CORONEL,NaN,NaN


In [45]:
##From the web page we import all the dataframe containing the offices of BBVA in Peru, the addresses, the region, etc.
bbva_offices = tabula.read_pdf("https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf", pages="all", stream=True)
bbva_offices[1].dropna(how='all', axis=1, inplace=True)
bbva_offices[0]= bbva_offices[0].iloc[2:, :]
for i in range(len(bbva_offices)):
    bbva_offices[i] = bbva_offices[i].reset_index().T.reset_index().T.reset_index()
    bbva_offices[i].drop(bbva_offices[i].iloc[:, 0:2], axis = 1, inplace=True)
    bbva_offices[i].set_axis(["OFICINA", "DIRECCION", "DISTRITO", "PROVINCIA", "DEPARTAMENTO"], axis=1, inplace=True)
bbva_offices = pd.concat(bbva_offices, ignore_index= True)
#For our excercise we only need Lima, so we just keep the offices thar are located in the department of Lima
bbva_offices_lima = bbva_offices.loc[bbva_offices["DEPARTAMENTO"] == "LIMA", :]
bbva_offices_lima

Got stderr: ene 26, 2024 4:39:47 P. M. org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

/var/folders/wh/bxfdtqcs1sg6fkjpzs6hq9v00000gn/T/ipykernel_96449/732955854.py:8: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  bbva_offices[i].set_axis(["OFICINA", "DIRECCION", "DISTRITO", "PROVINCIA", "DEPARTAMENTO"], axis=1, inplace=True)


ValueError: Length mismatch: Expected axis has 4 elements, new values have 5 elements

In [46]:
import tabula

# URL of the PDF
url = 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'

# Using tabula to read the PDF
# Set pages to 'all' to read all pages, or specify specific pages
# Since the structure of the table is unknown, lattice and stream modes are both tried
try:
    # Trying with lattice mode first
    tables = tabula.read_pdf(url, pages='all', lattice=True, multiple_tables=True)
except Exception as e:
    print(f"Lattice mode failed: {e}")
    try:
        # Trying with stream mode if lattice fails
        tables = tabula.read_pdf(url, pages='all', stream=True, multiple_tables=True)
    except Exception as e:
        print(f"Stream mode failed: {e}")
        tables = None

# Displaying the extracted tables
if tables:
    for i, table in enumerate(tables):
        print(f"Table {i+1}:")
        print(table)
else:
    print("No tables extracted.")

Got stderr: ene 26, 2024 4:42:24 P. M. org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



Table 1:
  Ubica tu ocina\rmás cercana.\rEscanea el\rCódigo QR  Unnamed: 0  Unnamed: 1  \
0                                                NaN          NaN         NaN   
1                                                NaN          NaN         NaN   
2                                         Escanea el          NaN         NaN   

   Unnamed: 2  
0         NaN  
1         NaN  
2         NaN  
Table 2:
   Ubica tu ocina\rmás cercana.\rEscanea el\rCódigo QR  Unnamed: 0  Unnamed: 1
0                                                NaN           NaN         NaN
Table 3:
   Ubica tu ocina\rmás cercana.\rEscanea el\rCódigo QR  Unnamed: 0
0                                                NaN           NaN
Table 4:
   Ubica tu ocina\rmás cercana.\rEscanea el\rCódigo QR  Unnamed: 0
0                                                NaN           NaN
Table 5:
  Ubica tu ocina\rmás cercana.\rEscanea el\rCódigo QR
0                                         Escanea el 
Table 6:
                       

In [47]:
link = 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'
bbva_dir = tabula.read_pdf(link, pages="all", stream=True, encoding='ISO-8859-1')
for page in range(0,len(bbva_dir)):
    # Obtener el número de columnas 
    n_var = len(bbva_dir[page].columns)
    
    # especificar limpieza para la primera página
    if page == 0:
        # Usamos la primera fila como nombres de las columnas
        bbva_dir[page].columns = bbva_dir[page].iloc[0]
        # Almacenar los nombres de las columnas en un objeto
        column_names = bbva_dir[page].columns
        # Eliminar la primera fila, que contiene los nombres de las columnas
        bbva_dir[page] = bbva_dir[page].drop(bbva_dir[page].index[0])
        # Almacenar el dataframe preprocesado en un nuevo df 
        bbva_offices = bbva_dir[page]
        # Resetear el índice del nuevo dataframe 
        bbva_offices.reset_index()
    # especificar limpieza si el número de columnas es igual a 6 
    elif n_var == 6:
        # Eliminar columna innecesaria
        bbva_dir[page] = bbva_dir[page].drop('Unnamed: 0', axis=1)
        row = list(bbva_dir[page].columns)
        bbva_dir[page].loc[len(bbva_dir[page])] = row
        bbva_dir[page].columns = column_names
        last_row = bbva_dir[page].iloc[-1]
        bbva_dir[page] = bbva_dir[page].iloc[0:-1]
        bbva_dir[page] = pd.concat([last_row.to_frame().T, bbva_dir[page]], ignore_index=True)
        bbva_offices = pd.concat([bbva_dir[page], bbva_offices])
        bbva_offices.reset_index()
    # 
    else:
        row = list(bbva_dir[page].columns)
        bbva_dir[page].loc[len(bbva_dir[page])] = row
        bbva_dir[page].columns = column_names
        last_row = bbva_dir[page].iloc[-1]
        bbva_dir[page] = bbva_dir[page].iloc[0:-1]
        bbva_dir[page] = pd.concat([last_row.to_frame().T, bbva_dir[page]], ignore_index=True)
        # Concatenar la última fila con el dataframe preprocesado anteriormente
        bbva_offices = pd.concat([bbva_dir[page], bbva_offices])
        bbva_offices.reset_index()

Got stderr: ene 26, 2024 4:44:57 P. M. org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [53]:
# Transformacion de PDF a DataFrame
file_path = 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'
pdf_data = tabula.read_pdf(file_path, columns=[0, 1, 2, 3, 4], pages='all',
                          pandas_options={'header':None})
# Se observa que el objeto pdf_data es una lista, cuyos elementos son DataFrame
print(pdf_data[1])

Got stderr: ene 26, 2024 4:52:53 P. M. org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



                                                    0          1  \
0                                             OFICINA  DIRECCION   
1                               APOLO AV. MEXICO 1700        NaN   
2                               CAPON JR. UCAYALI 735        NaN   
3   DAMERO GAMARRA AV. 28 DE JULIO 2498 Y AV. PROL...        NaN   
4                        GAMARRA PROLONG. UNANUE 1599        NaN   
5   HIPOLITO UNANUE JR HIPOLITO UNANUE 1616-1620, ...        NaN   
6                     LA VICTORIA AV. MANCO CAPAC 300        NaN   
7                               MEXICO AV. MEXICO 568        NaN   
8           NICOLAS ARRIOLA 1 AV. NICOLAS ARRIOLA 607        NaN   
9          NICOLAS ARRIOLA 2 AV. NICOLAS ARRIOLA 1799        NaN   
10  SEBASTIÁN BARRANCA AV. IQUITOS 795-799, ESQ. C...        NaN   
11                               BARRANCA 209-211-213        NaN   
12                    VENEZUELA II AV. VENEZUELA 1842        NaN   
13                    28 DE JULIO AV. 28 DE JULI

In [55]:
print(pdf_data[0])

                                                    0             1  \
0                                   OFICINA DIRECCION  DEPARTAMENTO   
1   AEROPUERTO CENTRO AÉREO COMERCIAL LOCALES 110 ...          LIMA   
2         BASE NAVAL AV. CTRMTE.  MORA S/N BASE NAVAL          LIMA   
3   C.C. CANTA CALLAO AV. ELMER FAUCETT Y ALEJANDR...          LIMA   
4                               CC CANTA CALLO, LC 13           NaN   
..                                                ...           ...   
59  EL REMANSO CA. LOS ALAMOS MZ C-2 LT. 5 - URB. ...          LIMA   
60  LA MOLINA AV. LA MOLINA MZ. B LOTE 08, URB. RE...          LIMA   
61  LA ROTONDA AV. LA FONTANA 371, MZ. A LT. 7, UR...          LIMA   
62                  RINCONADA AV. LA UNIVERSIDAD 1810          LIMA   
63            SEPARADORA INDUSTRIAL AV. LA MOLINA 540          LIMA   

            2          3  
0   PROVINCIA   DISTRITO  
1        LIMA     CALLAO  
2        LIMA     CALLAO  
3        LIMA     CALLAO  
4         Na